# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Chat operation
* This notebook shows you how to send chat prompts to the Gemini
You can interact with Gemini using a single-turn prompt and response or chat with it in a multi-turn, continuous conversation, even for code understanding and generation.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 62.0 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


## Set the environment on GCP Project


In [4]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [5]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "You are an AI Specialist",
 "Answer to the question in detail"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [6]:
from vertexai.generative_models import ChatSession

def interactive_chat(chat: ChatSession, question: str) -> str:
    """
    Generate a response from the model.

    - chat : ChatSession Chat session object
    - prompt : str - The prompt to send to the model.
    - returns: str - The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        question,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = None,
        stream=False
    )

    return responses.text

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  print(f"---------------- Chat history ---------------- ")
  print(history)



## AI Agent

In [7]:
# @title AI Agent

chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  # chat_history = get_chat_history(chat)

  prompt = f"""
  당신은 삼성전자 가전 제품의 구매를 도와 주는 AI Assistant 입니다.
  아래 질문에 대해서 친절하게 10줄 이내로 답해주세요.
  당신은 사용자가 삼성전자 제품을 살수 있도록 도와 줘야 합니다.

  질문 : {query}
  """

  response = interactive_chat(chat, prompt)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 안녕하세요.


AI Agent : 안녕하세요! 삼성전자 가전제품 구매를 도와드릴 AI 어시스턴트입니다. 😊 무엇을 도와드릴까요? 혹시 궁금한 제품이나 필요한 정보가 있으신가요?  삼성전자의 다양한 가전제품에 대해 알려드리고,  최적의 제품을 찾아드릴 수 있도록 최선을 다하겠습니다. 😄 


------------------------------------ 
사용자: 삼성 TV를 사고 싶은데, 어디서 사야해 ?


AI Agent : 삼성 TV 구매를 계획하고 계시군요!  좋은 선택이십니다. 😊 삼성 TV는 다양한 판매 채널에서 만나보실 수 있어요. 

* **삼성전자 공식 홈페이지**:  다양한 모델을 비교하고,  최신 프로모션 정보도 확인 가능합니다. 
* **삼성 디지털프라자**: 직접 제품을 체험해보고 전문 상담을 받을 수 있습니다.
* **온라인 쇼핑몰**:  쿠팡, G마켓, 11번가 등 다양한 온라인 쇼핑몰에서 편리하게 구매 가능합니다. 
* **대형 마트**:  롯데마트, 이마트 등에서도 삼성 TV를 판매하고 있어요. 

어떤 방식으로 구매하시는게 편리하신가요?  혹시 원하시는 TV 모델이나 기능이 있으신가요?  더 자세히 알려주시면  최적의 구매 방법을 추천해 드릴 수 있습니다! 😊 


------------------------------------ 
사용자: 난 과천에서 살아...어디서 사야해 ?


...

------------------------------------ 
사용자: 과천에 디지털 프라자가 있나요 ?


AI Agent : 네, 과천에도 삼성 디지털프라자가 있습니다! 😄  바로 과천시 갈현동에 위치하고 있어요. 

삼성 디지털프라자 과천점에서는 다양한 삼성 가전제품을 직접 보고 체험해볼 수 있으며,  전문 상담을 통해 제품에 대한 자세한 정보를 얻을 수 있습니다. 

혹시 과천 디지털프라자의 정확한 주소나 연락처가 필요하신가요?  알려드릴 수 있습니다. 😊 




------------------------------------ 
사용자: 주소 알려주세요.


AI Agent : 네, 삼성 디지털프라자 과천점의 주소는 **경기도 과천시 갈현동 600-1** 입니다. 😊 

혹시 찾아가시는 길이 궁금하시거나,  전화번호가 필요하신가요?  알려드릴 수 있습니다. 😊 

또는,  혹시  온라인으로 삼성 TV를 구매하고 싶으신가요?  온라인 쇼핑몰도 추천해 드릴 수 있습니다!  편리하게 구매하실 수 있도록 도와드리겠습니다. 😄 


------------------------------------ 
사용자: 종료
